Here is my previous code for agent :

In [ ]:
from random import random
def run():
    questions=['Q1', 'Q2', 'Q3', 'Q4']
    # Define the probabilities and rewards based on the inputA
    probabilities = [0.9, 0.75, 0.5, 0.1]
    rewards = [100, 1000, 10000, 50000]
    total_reward=0
    
    for k in range(len(questions)):
        Question = questions[k]
        Decision=input('Do you want to play? Y/N')
        while Decision not in ['Y','N']:
            print('Wrong entry, please answer Y or N')
            Decision=input('Do you want to play? Y/N')

        if Decision == 'Y': #then we play
            issue=random()
            if issue<probabilities[k]: #then you won
                print('You won the question',k+1)
                rew = rewards[k]
                total_reward+=rew
            else : #then you lost
                print('You lost')
                break
        else:
            print('Quit')
            break
    print('Your total rewards is',total_reward)

    


I did something really idiomatic firstly but lost a lot of time :

In [12]:
import numpy as np


def getNextstates(state,action):
    if state=='Q1':
        return(['Q1_win','Q1_fail'])
    if state=='Q2':
        if action=='play':
            return(['Q2_win','Q2_fail'])
        else :
            return(['quit'])
    if state=='Q3':
        if action=='play':
            return(['Q3_win','Q3_fail'])
        else:
            return(['quit'])
    if state=='Q4':
        if action=='play':
            return(['Q4_win','Q4_fail'])
        else :
            return(['quit'])


def getProbTr(state,next_state):
    trans_probabilities = [0.9, 0.75, 0.5, 0.1]
    if state=='Q1':
        if next_state=='Q1_win':
            return(0.9)
        elif next_state=='Q1_fail':
            return(0.1)
    if state=='Q2':
        if next_state=='Q2_win':
            return(0.75)
        elif next_state=='Q2_fail':
            return(0.25)
        elif next_state=='quit':
            return(1.0)
    if state=='Q3':
        if next_state=='Q3_win':
            return(0.5)
        elif next_state=='Q3_fail':
            return(0.5)
        elif next_state=='quit':
            return(1.0)
    if state=='Q4':
        if next_state=='Q4_win':
            return(0.9)
        elif next_state=='Q4_fail':
            return(0.1)
        elif next_state=='quit':
            return(1.0)

def getRewardVal(state,next_state):
    if state=='Q1':
        if next_state=='Q1_win':
            return(100)
        else: #you either fail or quit
            return(0)
    if state=='Q2':
        if next_state=='Q2_win':
            return(1100)
        else: #you either fail or quit
            return(100)
    if state=='Q3':
        if next_state=='Q3_win':
            return(11100)
        else: #you either fail or quit
            return(1100)
    if state=='Q4':
        if next_state=='Q4_win':
            return(61100)
        else: #you either fail or quit
            return(11100)
        
def getMaxVal(next_state):
    
    return(1) #for now
  

def best_policy():
    states=['Q1','Q2', 'Q3', 'Q4']
    actions=['quit','play']
    rewards = [100, 1000, 10000, 50000]
    trans_probabilities = [0.9, 0.75, 0.5, 0.1]
    q_matrix=np.zeros((4,2))
    print(q_matrix)
    convergence=False
    gamma=0.99
    while not convergence :
        new_q=np.zeros((4,2))
        for state in states:
            raw_idx=states.index(state)
            for action in actions:
                col_idx=actions.index(action)
                next_states=getNextstates(state,action)
                sum1=0
                for k in range(len(next_states)):
                    prob= getProbTr(state,next_states[k])
                    reward = getRewardVal(state,next_states[k])
                    max_val=getMaxVal(next_states[k])
                    sum1+=prob*(reward+gamma*max_val)
                new_q[raw_idx,col_idx] =sum1
        
        diff=0
        for state in states:
            raw_idx=states.index(state)
            for action in actions:
                col_idx=actions.index(action)
                diff+=abs(new_q[raw_idx,col_idx]-q_matrix[raw_idx,col_idx])
        if diff<0.5 :
            print('convergence')
            return(new_q)
            
        else :
            q_matrix=new_q
        

In [13]:
best_policy()

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
convergence


array([[   90.99,    90.99],
       [  100.99,   850.99],
       [ 1100.99,  6100.99],
       [11100.99, 56100.99]])

and I rewrote it using matrices, but obviously for the last state I can't compute the next state s' so my last raw of Q is empty: 

In [1]:
import numpy as np

def value_iteration(states,actions,gamma):
    R = np.array([    #reward matrix
        [100, 0],
        [1000, 100],
        [10000, 1000],
        [50000,10000]])
    P=np.array([       #it's the probabilities of transitions
        [0.9,0.1],
        [0.75,0.25],
        [0.5,0.5],
        [0.1,0.9]])
    Q = np.zeros((states,actions)) #this is the Q table

    convergence=False
    step=0
    while not convergence:
        last_Q = np.copy(Q) #last state of the Q matrix
        for state in range(states):   #to fill the Q matrix
            for action in range(actions):   #to fill the Q matrix
                sum1=0
                for state_prime in range(state+1,states):  #sum over the s'
                    best_action=np.zeros(actions)
                    for k in range(actions):  #looking for the argmax over a'
                        best_action[k]=last_Q[state_prime,k]
                    best=max(best_action)
                    sum1+=P[state_prime,action]*(R[state_prime,action]+gamma*best)
                Q[state,action]=sum1
        diff=0
        for action in range(actions):
            for state in range(states):
                diff+=abs(Q[state,action]-last_Q[state,action])
        if diff<0.01 :
            print('convergence')
            convergence=True
        else:
            step+=1
    print(Q)
value_iteration(states=4,actions=2,gamma=0.99)

convergence
[[25937.8375 17557.6125]
 [14455.     13955.    ]
 [ 5000.      9000.    ]
 [    0.         0.    ]]
